In [44]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
import requests
import time
from selenium.webdriver import Chrome
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re
# import parser
import datetime
from urllib.request import urlopen
import webbrowser
import xmltodict
import json
from ast import literal_eval
import lxml
import numpy as np
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36")

### 네이버 금융 주요뉴스 (제목, 일자, 신문사, 내용, 기자명, 링크)
url = "https://finance.naver.com/news/mainnews.nhn"

In [45]:
# 2005-09-30 ~ date_list 만들기
import time
from datetime import date,timedelta
today= date.today()
date_lst=[]
for i in range(5776):
    date_lst.append((today - timedelta(days=i)).strftime("%Y-%m-%d"))
len(date_lst),date_lst[0],date_lst[-1]

(5776, '2021-07-25', '2005-10-02')

In [46]:
dateurl_lst=[]
for i in range(5776):
    url = 'https://finance.naver.com/news/mainnews.nhn?date={}'.format(date_lst[i])
#     print(url)
    dateurl_lst.append(url)
print(len(dateurl_lst))
dateurl_lst[:5]

5776


['https://finance.naver.com/news/mainnews.nhn?date=2021-07-25',
 'https://finance.naver.com/news/mainnews.nhn?date=2021-07-24',
 'https://finance.naver.com/news/mainnews.nhn?date=2021-07-23',
 'https://finance.naver.com/news/mainnews.nhn?date=2021-07-22',
 'https://finance.naver.com/news/mainnews.nhn?date=2021-07-21']

In [ ]:
dateurl_lst

In [47]:
dateurl_lst[2335]

'https://finance.naver.com/news/mainnews.nhn?date=2015-03-04'

In [48]:
df=pd.DataFrame()

In [101]:
browser = Chrome()

In [102]:
for i in range(5084,len(dateurl_lst)):
    
    # 한 날짜의 링크모으기
    link_lst=[]
    for page in range(1,3): # 페이지
        pg_num = '&page='+str(page)    
        browser.get(dateurl_lst[i]+pg_num)
        result = browser.page_source
        soup = BeautifulSoup(result, 'html.parser')
        doc = soup.find_all('dt')
        for j in range(1,len(doc)):
            link_lst.append(doc[j].find('a').attrs['href'])
    print(str(i+1),'페이지 기사 개수',len(link_lst))

    # 링크마다 들어가서 정보 모으기    
    for k in range(len(link_lst)): # 날짜

        url = 'https://finance.naver.com/'+link_lst[k]
        try:
            browser.get(url)
            result = browser.page_source
            sour2 = BeautifulSoup(result, "html.parser")

            try:
                title = sour2.find_all("h3")[1].text.replace('\t',' ').replace('\n',' ').strip()
            except:
                title = np.nan
            try:
                date = sour2.find_all('span',{'class':'article_date'})[0].text
            except:
                date = np.nan
            try:
                kap = sour2.find_all('span',{'class':'press'})[0].find_all('img')[0]['title']
            except:
                kap = np.nan
            try:
                content = sour2.find_all('div',{'class':'articleCont'})[0].text.replace('\t',' ').replace('\n',' ').strip()
            except:
                content = np.nan
            try:
                reporter = sour2.find_all('div',{'class':'card_journalist_byline'})[0].text.replace('\t','').replace('\n','').strip()
            except:
                reporter = np.nan

            record  = pd.Series([title,date,kap,content,reporter,url],index = ['title','date','kap','content','reporter','link'])
    #         print(record)

            df = df.append(record, ignore_index = True)
        except:
            pass

    print(str(i+1),'페이지',str(k+1),'번째 기사 record',record)

5085 페이지 기사 개수 40
5085 페이지 40 번째 기사 record title                                 (투자의맥)"실적호조 대표株, 쌀때 사라"
date                                         2007-08-24 08:15
kap                                                      이데일리
content     [이데일리 유동주기자] 김중현 굿모닝신한증권 연구원은 "단기적으로 현상황에서 최대 ...
reporter                                      유동주  (law1123@)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5086 페이지 기사 개수 40
5086 페이지 40 번째 기사 record title                                亞 증시, 신용경색 완화에 `일제히 급등세`
date                                         2007-08-23 10:38
kap                                                      매일경제
content     아시아 증시가 신용 경색 우려 완화와 미 증시의 강세로 일제히 상승세를 보이고 있다...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5087 페이지 기사 개수 40
5087 페이지 40 번째 기사 record title                                          코스피, 3일째 반등 지속
date                                 

5105 페이지 기사 개수 8
5105 페이지 8 번째 기사 record title                              <증시전망> 변동성 확대 지속..FOMC가 변수
date                                         2007-08-04 11:16
kap                                                      연합뉴스
content     (서울=연합뉴스) 이웅 기자 = 주식시장의 변동성이 커지고 있다. 이번 주  증시는...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5106 페이지 기사 개수 40
5106 페이지 40 번째 기사 record title                                  (투자의맥)"본격 반등은 시간이 필요해"
date                                         2007-08-03 07:46
kap                                                      이데일리
content     [이데일리 손희동기자] 지수선물은 전일 외국인이 모처럼 순매수로 돌아서며 반등에 성...
reporter                                     손희동  (sonnycap@)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5107 페이지 기사 개수 40
5107 페이지 40 번째 기사 record title                                     건설-IT株, 코스피 상승 `견인`
date                                   

5125 페이지 기사 개수 20
5125 페이지 20 번째 기사 record title                                  돈의 힘 "코스피 2000p도 한달음에"
date                                         2007-07-15 09:03
kap                                                      이데일리
content     - (주간증시 전망)"조정압력 커졌지만 팔지는 말라"[이데일리 오상용기자] 멀게만 ...
reporter                                       오상용  (thugoh@)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5126 페이지 기사 개수 30
5126 페이지 30 번째 기사 record title                                          주가 연중최고 53P 폭등
date                                         2007-07-14 02:21
kap                                                      서울신문
content     [서울신문]주가가 1950을 단숨에 넘어섰다. 환율은 연중 최저지를 기록했다. 미국...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5127 페이지 기사 개수 40
5127 페이지 40 번째 기사 record title                                         증시 뜨겁지만 버블은 아니다
date                                 

5145 페이지 기사 개수 40
5145 페이지 40 번째 기사 record title                                    `중소형 건설株 선별투자 필요`-대신
date                                         2007-06-25 08:41
kap                                                      이데일리
content     [이데일리 이진철기자] 대신증권은 25일 건설업종에 대해 "대형사 위주의 주가 레벨...
reporter                                        이진철  (cheol@)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5146 페이지 기사 개수 16
5146 페이지 16 번째 기사 record title                                             차기 주도주를 찾아라
date                                         2007-06-24 07:20
kap                                                      이데일리
content     - (주간추천종목)실적개선 기대 LG·삼성 계열사 두각- 증권주 인기 여전..코스닥...
reporter                                         이대희  (eday@)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5147 페이지 기사 개수 2
5147 페이지 2 번째 기사 record title                                <주요 종목 주식 `품귀’…주가 상승 예고>
date                                   

5165 페이지 기사 개수 40
5165 페이지 40 번째 기사 record title                                      "中조정에도 상승은 유효"-삼성證
date                                         2007-06-05 09:08
kap                                                     머니투데이
content     [머니투데이 오상연 기자]삼성증권은 5일 중국 증시는 조정을 받고 있지만 코스피 지...
reporter                                                  오상연
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5166 페이지 기사 개수 40
5166 페이지 40 번째 기사 record title                          [웰빙 포트폴리오] "고점 멀었다… 길게보고 가져가라"
date                                         2007-06-04 11:17
kap                                                      서울경제
content     ‘대세 상승기에는 길게 보고 가져가라.’ 주식시장의 강세흐름이 이어지면서 시장의 관...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5167 페이지 기사 개수 22
5167 페이지 22 번째 기사 record title                               변동성 큰 장세 예상...IT 저가 매수 기회
date                                 

...

5685 페이지 기사 개수 20
5685 페이지 20 번째 기사 record title                                       업종별 대표주 자리바꿈 '활발'
date                                         2006-01-01 07:02
kap                                                      연합뉴스
content     (서울=연합뉴스) 고미혜 기자 = 강세장이 펼쳐졌던 2005년  한해  주식시장에서...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5686 페이지 기사 개수 10
5686 페이지 10 번째 기사 record title                                        (월가시각)희망을 잉태한 실망
date                                         2005-12-31 07:05
kap                                                      이데일리
content     [뉴욕=이데일리 안근모특파원] "산타클로스는 결국 오지 않았다." 일반적으로 말하는...
reporter                            안근모  (ahnkm@edaily.co.kr)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5687 페이지 기사 개수 21
5687 페이지 21 번째 기사 record title                               [코스닥시황] 기관 순매수로 10P올라 701
date                                 

5705 페이지 기사 개수 34
5705 페이지 34 번째 기사 record title                                    (투자의맥)증시, 변동성 확대 불가피
date                                         2005-12-12 08:15
kap                                                      이데일리
content     [이데일리 김춘동기자] 김성주 대우증권 연구원은 12일 "이번 주에는 주식시장의 변...
reporter                             김춘동  (bomy@edaily.co.kr)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5706 페이지 기사 개수 14
5706 페이지 14 번째 기사 record title                                [코스닥전망]종목간 차별화 심화..조정 우세
date                                         2005-12-11 14:45
kap                                                     머니투데이
content     [머니투데이 이웅 기자]이번 주(12~16일) 코스닥시장은 추가 상승보다는 조정 쪽...
reporter                                                   이웅
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5707 페이지 기사 개수 0
5707 페이지 14 번째 기사 record title                                [코스닥전망]종목간 차별화 심화..조정 우세
date                                  

5725 페이지 기사 개수 36
5725 페이지 36 번째 기사 record title                                         [원포인트]강세장에 올라타라
date                                         2005-11-22 07:15
kap                                                     머니투데이
content     [머니투데이 전병서 대우증권 상무/리서치센터장]주가가 신고치를 갱신하고 난 이후 숨...
reporter                                                  전병서
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5726 페이지 기사 개수 29
5726 페이지 29 번째 기사 record title                                 경영참가목적 외국인투자가 차익 실현 잇따라
date                                         2005-11-21 06:06
kap                                                      연합뉴스
content     M&A 기대감 심어 주가 올린 뒤 빠지기 비판도     (서울=연합뉴스) 김대호 김...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5727 페이지 기사 개수 10
5727 페이지 10 번째 기사 record title                                      상승 피로도 vs 연말랠리 기대감
date                                 

5745 페이지 기사 개수 25
5745 페이지 25 번째 기사 record title                                      은행주가 이끄는 장세 얼마나 갈까
date                                         2005-11-02 06:40
kap                                                      이데일리
content     [이데일리 김춘동기자] 주식시장의 분위기가 확 달라졌다. 지난 주 1100선 붕괴를...
reporter                             김춘동  (bomy@edaily.co.kr)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5746 페이지 기사 개수 40
5746 페이지 40 번째 기사 record title                               [원포인트]주가, 연준의 금리인상 이미 선반영
date                                         2005-11-01 07:30
kap                                                     머니투데이
content     [머니투데이 전병서 대우증권 상무/리서치본부장]주가는 환율, 유가, 금리 등의 가격...
reporter                                                  전병서
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5747 페이지 기사 개수 25
5747 페이지 25 번째 기사 record title                                  (투자의맥)기술적 반등이 예상되는 한 주
date                                 

5765 페이지 기사 개수 28
5765 페이지 28 번째 기사 record title                                      노무라,한국 강세장 아직 안끝났다
date                                         2005-10-13 06:59
kap                                                      한국경제
content     노무라증권이 한국 증시에 대해 주가수익배율(PER) 팽창이 이어질 것으로 전망하는 ...
reporter                                                  NaN
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5766 페이지 기사 개수 30
5766 페이지 30 번째 기사 record title                                   미수금 연일 사상최고..증시 과열신호?
date                                         2005-10-12 06:30
kap                                                      이데일리
content     [이데일리 김춘동기자] 주가가 연일 오르면서 돈을 빌려서 주식을 산 미수금 규모가 ...
reporter                             김춘동  (bomy@edaily.co.kr)
link        https://finance.naver.com//news/news_read.nhn?...
dtype: object
5767 페이지 기사 개수 29
5767 페이지 29 번째 기사 record title                                     [원포인트]조정은 짧고 상승은 길다
date                                 

In [92]:
html2=urlopen(url)

In [21]:
df_save = df

In [36]:
df_save.shape

NameError: name 'df_save' is not defined

In [37]:
df.shape

(22, 6)

In [104]:
df.shape

(107495, 6)

In [110]:
df.drop_duplicates().shape

(97078, 6)

In [28]:
df_save = pd.concat([df_save,a],axis=0).drop_duplicates()

### DF 저장

In [27]:
# os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data')
os.getcwd()

'C:\\Users\\shic\\Documents\\GitHub\\Crawling\\Desktop\\Project'

In [106]:
df.to_csv('1_Data\\naver_finance_Major_news_7.csv',index=False)

In [30]:
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project'

In [11]:
a = pd.read_csv('1_Data\\naver_finance_major_news_3.csv')
b = pd.read_csv('1_Data\\naver_finance_Major_news_4.csv')
a.shape,b.shape

((20457, 6), (36152, 6))

In [13]:
ab = pd.concat([a,b],axis=0).drop_duplicates()
print(ab.shape)
ab.to_csv('1_Data\\naver_finance_Major_news_5.csv',index=False)

(36152, 6)


In [38]:
ab.date.sort_values()

15726    2015-03-02 15:42
15729    2015-03-02 16:47
15728    2015-03-02 17:24
15727    2015-03-02 17:58
15725    2015-03-03 07:39
               ...       
14556                 NaN
14678                 NaN
14863                 NaN
15188                 NaN
15568                 NaN
Name: date, Length: 36152, dtype: object

In [25]:
a.head()

,content,date,kap,link,reporter,title
0,[서울경제] 코스피가 23일 강보합으로 출발했으나 이내 하락 전환했다.이날 오전 9...,2021-07-23 09:12,서울경제,https://finance.naver.com//news/news_read.nhn?...,양사록 기자(sarok@sedaily.com),"[오전시황] 강보합 출발한 코스피, 이내 하락 전환"
1,"델타 변이 확산에도 국채금리 상승이동성 지표 코로나19 이전 수준 회복美 증시, 인...",2021-07-23 08:32,아시아경제,https://finance.naver.com//news/news_read.nhn?...,이민지(ming@asiae.co.kr),"[굿모닝 증시] ""델타 변이, 시장에 큰 충격 주진 못할 것"""
2,"ECB 기준금리 동결, 인플레 목표치 상향유로화 약세에 달러인덱스 92수준 강보합위...",2021-07-23 08:15,이데일리,https://finance.naver.com//news/news_read.nhn?...,이윤화(akfdl34@edaily.co.kr),"[외환브리핑]유로화 약세에 강달러..환율, 1150원 재탈환 예상"
3,[골드스미스=AP/뉴시스]21일(현지시간) 미 텍사스주 골드스미스 인근 유정의 원유...,2021-07-23 08:08,파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...,서혜진(sjmary@fnnews.com),"[원자재시황] 국제유가, 공급부족 우려에 상승..WTI 2.3%↑"
4,(출처=뉴시스/NEWSIS) /사진=뉴시스 [파이낸셜뉴스] 유럽 주요 증시는 22...,2021-07-23 07:37,파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...,서혜진(sjmary@fnnews.com),[유럽증시] ECB 금리동결에 대부분 상승


In [9]:
a

,content,date,kap,link,reporter,title
0,[서울경제] 코스피가 23일 강보합으로 출발했으나 이내 하락 전환했다.이날 오전 9...,2021-07-23 09:12,서울경제,https://finance.naver.com//news/news_read.nhn?...,양사록 기자(sarok@sedaily.com),"[오전시황] 강보합 출발한 코스피, 이내 하락 전환"
1,"델타 변이 확산에도 국채금리 상승이동성 지표 코로나19 이전 수준 회복美 증시, 인...",2021-07-23 08:32,아시아경제,https://finance.naver.com//news/news_read.nhn?...,이민지(ming@asiae.co.kr),"[굿모닝 증시] ""델타 변이, 시장에 큰 충격 주진 못할 것"""
2,"ECB 기준금리 동결, 인플레 목표치 상향유로화 약세에 달러인덱스 92수준 강보합위...",2021-07-23 08:15,이데일리,https://finance.naver.com//news/news_read.nhn?...,이윤화(akfdl34@edaily.co.kr),"[외환브리핑]유로화 약세에 강달러..환율, 1150원 재탈환 예상"
3,[골드스미스=AP/뉴시스]21일(현지시간) 미 텍사스주 골드스미스 인근 유정의 원유...,2021-07-23 08:08,파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...,서혜진(sjmary@fnnews.com),"[원자재시황] 국제유가, 공급부족 우려에 상승..WTI 2.3%↑"
4,(출처=뉴시스/NEWSIS) /사진=뉴시스 [파이낸셜뉴스] 유럽 주요 증시는 22...,2021-07-23 07:37,파이낸셜뉴스,https://finance.naver.com//news/news_read.nhn?...,서혜진(sjmary@fnnews.com),[유럽증시] ECB 금리동결에 대부분 상승
...,...,...,...,...,...,...
20452,"[아시아경제 임혜선 기자]코스피가 3일연속 하락 마감했다. 다만 삼성전자, SK 하...",2017-09-22 07:23,아시아경제,https://finance.naver.com//news/news_read.nhn?...,임혜선lhsro@asiae.co.kr,"[굿모닝 증시]코스피, '제2의 삼성전자'를 찾아라"
20453,[머니투데이 뉴욕(미국)=송정렬 특파원] 뉴욕증시가 하락했다. 3대 주요 지수가 일...,2017-09-22 06:27,머니투데이,https://finance.naver.com//news/news_read.nhn?...,뉴욕(미국)=송정렬,"[뉴욕마감]3대 지수, '매파' 연준에 하루 만에 일제히 하락"
20454,뉴욕증권거래소(NYSE). © AFP=뉴스1(서울=뉴스1) 이창규 기자 = 뉴욕증시...,2017-09-21 23:47,뉴스1,https://finance.naver.com//news/news_read.nhn?...,이창규 기자(yellowapollo@news1.kr),[뉴욕개장] 하락세…FOMC의 매파적 태도에 투자심리 위축
20455,금리인상 기대에 금융株↑ 알려진 악재는 역시 악재가 아니었다. 미국 연방준비제도(F...,2017-09-21 17:51,매일경제,https://finance.naver.com//news/news_read.nhn?...,최재원,시나리오대로 돈줄 죈 美…코스피는 `차분한 반응`


## 통합

In [108]:
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project'

In [ ]:
DF1 = pd.read_csv('1_Data\\naver_finance_major_news.csv')
DF2 = pd.read_csv('1_Data\\naver_finance_major_news_2.csv')
DF3 = pd.read_csv('1_Data\\naver_finance_major_news_3.csv')
DF4 = pd.read_csv('1_Data\\naver_finance_Major_news_4.csv')
DF5 = pd.read_csv('1_Data\\naver_finance_Major_news_5.csv')
DF6 = pd.read_csv('1_Data\\naver_finance_Major_news_6.csv')
DF7 = pd.read_csv('1_Data\\naver_finance_Major_news_7.csv')


In [ ]:
DF1.shape,DF2.shape,DF3.shape,DF4.shape,DF5.shape,DF6.shape,DF7.shape